## 測試項目
1. 倒水聲(15)及馬桶沖水聲(18)
2. 使用acdnet20_20khz_fold4 keras h5-format model
3. export to cc file

In [1]:
import os
import sys
import datetime
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
import zipfile
import wavio
from common import utils as U

### Loading Keras Model

In [2]:
#ref: https://stackoverflow.com/questions/50443411/how-to-load-a-tflite-model-in-script/51986982#51986982
"""
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)
"""

'\nimport numpy as np\nimport tensorflow as tf\n\n# Load TFLite model and allocate tensors.\ninterpreter = tf.lite.Interpreter(model_path="converted_model.tflite")\ninterpreter.allocate_tensors()\n\n# Get input and output tensors.\ninput_details = interpreter.get_input_details()\noutput_details = interpreter.get_output_details()\n\n# Test model on random input data.\ninput_shape = input_details[0][\'shape\']\ninput_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)\ninterpreter.set_tensor(input_details[0][\'index\'], input_data)\n\ninterpreter.invoke()\n\n# The function `get_tensor()` returns a copy of the tensor data.\n# Use `tensor()` in order to get a pointer to the tensor.\noutput_data = interpreter.get_tensor(output_details[0][\'index\'])\nprint(output_data)\n'

## Locading TFlite Model

In [3]:
tflite_model_path = "./th/tflite_models/acdnet_95.tflite"
net_interpreter = tf.lite.Interpreter(tflite_model_path)
net_interpreter.allocate_tensors()

In [26]:
# Get input and output tensors.
input_details = net_interpreter.get_input_details()
for i in input_details:
    print(f"{i},")
output_details = net_interpreter.get_output_details()

{'name': 'x.3', 'index': 17, 'shape': array([    1,     1, 30225,     1]), 'shape_signature': array([    1,     1, 30225,     1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}},


In [20]:
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
net_interpreter.set_tensor(input_details[0]['index'], input_data)


In [21]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [22]:
net_interpreter.invoke()
output_data = net_interpreter.get_tensor(output_details[0]['index'])
print(output_data.T)

[[0.218458]
 [0.184880]
 [0.156462]
 [0.127002]
 [0.150069]
 [0.163129]]


In [23]:
help(net_interpreter)
#net_interpreter.area_used_bytes()
print(net_interpreter.arean_szie)

Help on Interpreter in module tensorflow.lite.python.interpreter object:

class Interpreter(builtins.object)
 |  Interpreter(model_path=None, model_content=None, experimental_delegates=None, num_threads=None, experimental_op_resolver_type=<OpResolverType.AUTO: 0>, experimental_preserve_all_tensors=False, experimental_disable_delegate_clustering=False)
 |  
 |  Interpreter interface for running TensorFlow Lite models.
 |  
 |  Models obtained from `TfLiteConverter` can be run in Python with
 |  `Interpreter`.
 |  
 |  As an example, lets generate a simple Keras model and convert it to TFLite
 |  (`TfLiteConverter` also supports other input formats with `from_saved_model`
 |  and `from_concrete_function`)
 |  
 |  >>> x = np.array([[1.], [2.]])
 |  >>> y = np.array([[2.], [4.]])
 |  >>> model = tf.keras.models.Sequential([
 |  ...           tf.keras.layers.Dropout(0.2),
 |  ...           tf.keras.layers.Dense(units=1, input_shape=[1])
 |  ...         ])
 |  >>> model.compile(optimizer='s

AttributeError: 'Interpreter' object has no attribute 'arean_szie'

### Read Test Wav File

In [28]:
test_sound_file = "model_evalution_testsets/pouring_water/5-207781-A-17.wav"
# ec50_18_sound = "./test_sounds/toilet_flushing/1-20736-A-18.wav"

sound = wavio.read(test_sound_file).data.T[0]
start = sound.nonzero()[0].min()
end = sound.nonzero()[0].max()
sound = sound[start: end + 1]  # Remove silent sections
label = 18 #int(os.path.splitext(test_sound_file)[0].split('-')[-1])

if len(sound)> 220500:
    sound = sound[:220500]
"""
ec50_sound1 =  wavio.read(ec50_18_sound).data.T[0]
start_ec50 = ec50_sound1.nonzero()[0].min()
end_ec50 = ec50_sound1.nonzero()[0].max()
ec50_sound1 = ec50_sound1[start_ec50:end_ec50+1]
ec50_18_label = 18
"""


'\nec50_sound1 =  wavio.read(ec50_18_sound).data.T[0]\nstart_ec50 = ec50_sound1.nonzero()[0].min()\nend_ec50 = ec50_sound1.nonzero()[0].max()\nec50_sound1 = ec50_sound1[start_ec50:end_ec50+1]\nec50_18_label = 18\n'

In [30]:
print(sound)
print(len(sound))
# print(ec50_sound1)
# print(len(ec50_sound1))

[-230 -230 -221 ...    0    0   -1]
57072


In [38]:
_inputLen = 30225
_nCrops = 10
def preprocess_setup():
    funcs = []
    funcs += [U.padding( _inputLen// 2),
              U.normalize(32768.0),
              U.multi_crop(_inputLen, _nCrops)]

    return funcs

def preprocess(sound, funcs):
    for f in funcs:
        sound = f(sound)

    return sound;

In [39]:
_funcs = preprocess_setup()

In [42]:
# sound = preprocess(sound, _funcs)
# print(sound.shape)

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [34]:
sound = np.expand_dims(sound, axis=1)
sound = np.expand_dims(sound, axis=3)
print(sound.shape)

(10, 1, 30225, 1)


In [62]:
scores = model.predict(sound, batch_size=len(sound), verbose=0);
print(type(scores))
print(scores.shape)

for res in scores:
    max_value = res.max()
    max_index = np.argmax(res)
    print(f"max value:{max_value:.5f} and index is {max_index}")
    print('\n'.join('{}: {:.5f}'.format(*k) for k in enumerate(res)))

### loading dataset from npz format
1. ACDNet input length is 30225
2. sr is 44100 and 20000
3. need to convert 16K to 20000
### ACDNet Config Setting
#### Training Parameters
1. opt.batchSize = 64;
2. opt.weightDecay = 5e-4;
3. opt.momentum = 0.9;
4. opt.nEpochs = 2000;
5. opt.LR = 0.1;
6. opt.schedule = [0.3, 0.6, 0.9];
7. opt.warmup = 10; 
#### Basic Net Configuration
- nClasses = 50
- nFolds = 5
- splits = \[i for in range(1, nFolds + 1)\]
- sr = 20000
- inputLength = 30225
<br>ngth = 30225;
### How to convert 16K sound to 44.1K with python and sox

if using sox the command is as following: <br />
    sox old.wav -b 16 new.wav 
if using python you can do as following: <br />
    import soundfile
    
data, samplerate = soundfile.read('old.wav
    <br />)
soundfile.write('new.wav', data, samplerate, subtype='PCM_1
6')

In [4]:
### convert to 16-bit
# def convertAllFilesInDirectoryTo16Bit(directory):
#     for file in os.listdir(directory):
#          if(file.endswith('.wav')):
#              nameSolo = file.rsplit('.', 1)[0]
#              print(directory + nameSolo )
#              data, samplerate = soundfile.read(directory + file)                

#            soundfile.write('/Users/yournamehere/Desktop/folderwhereyouwanttosae/' + nameSolo + '16BIT.wav', data, samplerate, subtype='PCM_16')
#             print("converting " + file + "to 16 - bit")

In [5]:
def npz_headers(npz):
    """
    Takes a path to an .npz file, which is a Zip archive of .npy files.
    Generates a sequence of (name, shape, np.dtype).
    """
    with zipfile.ZipFile(npz) as archive:
        for name in archive.namelist():
            if not name.endswith('.npy'):
                continue

            npy = archive.open(name)
            version = np.lib.format.read_magic(npy)
            shape, fortran, dtype = np.lib.format._read_array_header(npy, version)
            yield name[:-4], shape, dtype

In [8]:
test_data = np.load("./datasets/esc50/test_44khz/", allow_pickle=True);
# test_20_X = data['x'];
# test_20_Y = data['y'];
list(data.keys())
data['fold1'].size
data['fold1'].shape
fold1_sounds_list = data['fold1'].item()['sounds']
fold1_labels_list = data['fold1'].item()['labels']
type(fold1_sounds_list)
len(fold1_labels_list)
fold1_sounds_ary = np.asarray(fold1_sounds_list,dtype="object") #set dtype="ojbect" is workable here
fold1_sounds_ary[0]

PermissionError: [Errno 13] Permission denied: './datasets/esc50/test_44khz/'

In [6]:
print(list(npz_headers("./datasets/esc50/wav44.npz")))

[('fold1', (), dtype('O')), ('fold2', (), dtype('O')), ('fold3', (), dtype('O')), ('fold4', (), dtype('O')), ('fold5', (), dtype('O'))]


In [7]:
data = np.load("./datasets/esc50/wav44.npz", allow_pickle=True);
# test_20_X = data['x'];
# test_20_Y = data['y'];
list(data.keys())
data['fold1'].size
data['fold1'].shape
fold1_sounds_list = data['fold1'].item()['sounds']
fold1_labels_list = data['fold1'].item()['labels']
type(fold1_sounds_list)
len(fold1_labels_list)
fold1_sounds_ary = np.asarray(fold1_sounds_list,dtype="object") #set dtype="ojbect" is workable here
fold1_sounds_ary[0]
# data['fold1'].item()['labels']

array([-1,  0,  0, ...,  0,  0, -1], dtype=int16)

(10, 30225)


(10, 1, 30225, 1)


<class 'numpy.ndarray'>
(10, 50)
max value:0.14976 and index is 48
0: 0.02089
1: 0.00920
2: 0.01522
3: 0.01064
4: 0.01116
5: 0.01254
6: 0.00811
7: 0.01380
8: 0.00581
9: 0.00685
10: 0.00122
11: 0.00854
12: 0.00112
13: 0.00187
14: 0.00439
15: 0.07874
16: 0.00267
17: 0.01940
18: 0.01060
19: 0.02611
20: 0.00529
21: 0.04162
22: 0.00808
23: 0.01187
24: 0.02780
25: 0.01830
26: 0.01558
27: 0.00153
28: 0.03831
29: 0.12634
30: 0.05225
31: 0.03831
32: 0.00732
33: 0.09516
34: 0.01023
35: 0.00628
36: 0.00490
37: 0.00652
38: 0.01954
39: 0.00562
40: 0.00611
41: 0.00352
42: 0.00272
43: 0.00618
44: 0.00310
45: 0.00521
46: 0.00440
47: 0.00405
48: 0.14976
49: 0.00520
max value:0.21250 and index is 45
0: 0.00370
1: 0.00382
2: 0.00763
3: 0.02212
4: 0.00357
5: 0.00874
6: 0.00334
7: 0.02871
8: 0.00756
9: 0.00562
10: 0.00119
11: 0.00901
12: 0.00143
13: 0.00085
14: 0.00566
15: 0.01414
16: 0.02045
17: 0.00054
18: 0.00634
19: 0.07517
20: 0.01037
21: 0.01930
22: 0.00741
23: 0.00719
24: 0.14637
25: 0.04677
26: 0.0